# MACS 40200
# Problem Set 3
## Yuyao Wu

In [219]:
import numpy as np
import scipy.stats as sts
import scipy.special as spec
import matplotlib.pyplot as plt
import scipy.optimize as opt
import scipy.integrate as integral

cons, totsav, wage, intrate = np.loadtxt('MacroSeries.txt', delimiter=',', unpack=True)

# (a).

In [220]:
#Firstly, use w, k, alpha, and equation (3) to express z.
def get_z(k, w, alpha):
    z = np.log(w) - np.log(1-alpha) - alpha*np.log(k)
    
    return z

#Then use equation 5 to get error
def get_epsilon(z, rho, mu):
    N = len(cons)
    epsilon = np.zeros(N)
    epsilon[0] = z[0] - mu
    for i in range(N-1):
        epsilon[i+1] = z[i+1] - rho * z[i] - (1-rho)*mu
        
    return epsilon


def norm_pdf(k, w, alpha, rho, mu, sigma):
    z = get_z(k, w, alpha)
    eps = get_epsilon(z, rho, mu)
    pdf_vals = np.maximum((1/np.sqrt(2*np.pi*(sigma**2))) * np.exp(-(eps**2)/(2*(sigma**2))), 1e-20)
    
    return pdf_vals

def log_lik_norm(k, w, alpha, rho, mu, sigma):
    pdf_vals = norm_pdf(k, w, alpha, rho, mu, sigma)
    ln_pdf_vals = np.log(pdf_vals)
    log_lik_val = ln_pdf_vals.sum()
    
    return log_lik_val

def crit(params, *args):
    alpha, rho, mu, sigma = params
    k, w = args
    log_lik_val = log_lik_norm(k, w, alpha, rho, mu, sigma)
    neg_log_lik_val = -log_lik_val
    
    return neg_log_lik_val

alpha_init = 0.4
rho_init = -0.5
mu_init = 4
sigma_init = 1.5
params_init = np.array([alpha_init, rho_init, mu_init, sigma_init])
mle_args = (totsav, wage)
results = opt.minimize(crit, params_init, args=(mle_args), method='L-BFGS-B', bounds=((1e-10, 1-1e-10), (-1+1e-10, 1-1e-10), (1e-10, None), (1e-10, None)))
alpha_mle, rho_mle, mu_mle, sigma_mle = results.x
HessianInv = results.hess_inv.todense()
print ('alpha =', alpha_mle, 'rho =', rho_mle, 'mu =', mu_mle, 'sigma =', sigma_mle)
print ('The value of the maximized log likelihood function is', log_lik_norm(totsav, wage, alpha_mle, rho_mle, mu_mle, sigma_mle))
print ('The corresponding Hessian Matrix is:')
print (HessianInv)

alpha = 0.702042616675 rho = 0.479861663257 mu = 6.2546870096 sigma = 0.0920456697135
The value of the maximized log likelihood function is 96.6537374993
The corresponding Hessian Matrix is:
[[  1.50959352e+02  -2.01510158e+02  -1.85322793e+03  -7.20174412e-01]
 [ -2.01510158e+02   2.72608474e+02   2.47345635e+03   1.04279147e+00]
 [ -1.85322793e+03   2.47345635e+03   2.27508858e+04   8.83316995e+00]
 [ -7.20174412e-01   1.04279147e+00   8.83316995e+00   5.32494489e-03]]


# (b).

In [221]:
#Firstly, use equation (4) to get z
def get_zb(k, r, alpha):
    z = np.log(r) - np.log(alpha) - (alpha - 1)*np.log(k)
    
    return z

#Then use equation 5 to get error
def get_epsilonb(z, rho, mu):
    N = len(cons)
    epsilon = np.zeros(N)
    epsilon[0] = z[0] - mu
    for i in range(N-1):
        epsilon[i+1] = z[i+1] - rho * z[i] - (1-rho)*mu
        
    return epsilon


def norm_pdfb(k, r, alpha, rho, mu, sigma):
    z = get_zb(k, r, alpha)
    eps = get_epsilonb(z, rho, mu)
    pdf_vals = np.maximum((1/np.sqrt(2*np.pi*(sigma**2))) * np.exp(-(eps**2)/(2*(sigma**2))), 1e-20)
    
    return pdf_vals

def log_lik_normb(k, r, alpha, rho, mu, sigma):
    pdf_vals = norm_pdfb(k, r, alpha, rho, mu, sigma)
    ln_pdf_vals = np.log(pdf_vals)
    log_lik_val = ln_pdf_vals.sum()
    
    return log_lik_val

def critb(params, *args):
    alpha, rho, mu, sigma = params
    k, r = args
    log_lik_val = log_lik_normb(k, r, alpha, rho, mu, sigma)
    neg_log_lik_val = -log_lik_val
    
    return neg_log_lik_val

alpha_init = 0.3
rho_init = -0.9
mu_init = 2
sigma_init = 4
params_init = np.array([alpha_init, rho_init, mu_init, sigma_init])
mle_args = (totsav, intrate)
results = opt.minimize(critb, params_init, args=(mle_args), method='L-BFGS-B', bounds=((1e-10, 1-1e-10), (-1+1e-10, 1-1e-10), (1e-10, None), (1e-10, None)))
alpha_mleb, rho_mleb, mu_mleb, sigma_mleb = results.x
HessianInvb = results.hess_inv.todense()
print ('alpha =', alpha_mleb, 'rho =', rho_mleb, 'mu =', mu_mleb, 'sigma =', sigma_mleb)
print ('The value of the maximized log likelihood function is', log_lik_normb(totsav, intrate, alpha_mleb, rho_mleb, mu_mleb, sigma_mleb))
print ('The corresponding Hessian Matrix is:')
print (HessianInvb)

alpha = 0.701847854588 rho = 0.480092809516 mu = 5.07822562669 sigma = 0.0920463476278
The value of the maximized log likelihood function is 96.6537367138
The corresponding Hessian Matrix is:
[[  5.27573532e+01  -4.24996263e+01  -9.17899890e+02   1.20590631e+01]
 [ -4.24996263e+01   3.42680304e+01   7.39444467e+02  -9.73737986e+00]
 [ -9.17899890e+02   7.39444467e+02   1.59701119e+04  -2.09827527e+02]
 [  1.20590631e+01  -9.73737987e+00  -2.09827527e+02   2.78383955e+00]]


# (c).

We want to solve for $P(r_{t}>1 |\hat{\theta},k_{t},z_{t-1})$

According to equation (4), this is equivalent to solve for $P(\alpha e^{z_{t}}k_{t}^{\alpha - 1}>1 |\hat{\theta},k_{t},z_{t-1})$

Then we have: $P(r_{t}>1 |\hat{\theta},k_{t},z_{t-1})$ = $P(\alpha e^{z_{t}}k_{t}^{\alpha - 1}>1 |\hat{\theta},k_{t},z_{t-1})$ = $P(e^{z_{t}}>\frac{1}{\alpha k_{t}^{\alpha - 1}} |\hat{\theta},k_{t},z_{t-1})$

So, $P(r_{t}>1 |\hat{\theta},k_{t},z_{t-1})$ = $P(z_{t}>\log{\frac{1}{\alpha k_{t}^{\alpha - 1}}} |\hat{\theta},k_{t},z_{t-1})$

$P(r_{t}>1 |\hat{\theta},k_{t},z_{t-1})$ = $P(z_{t}>-\log{\alpha} + (1-\alpha)\log{k_{t}} |\hat{\theta},k_{t},z_{t-1})$

According to equation (5), we have $P(r_{t}>1 |\hat{\theta},k_{t},z_{t-1})$ = $P(\rho z_{t-1} + (1-\rho)\mu + \epsilon_{t}>-\log{\alpha} + (1-\alpha)\log{k_{t}} |\hat{\theta},k_{t},z_{t-1})$

So, $P(r_{t}>1 |\hat{\theta},k_{t},z_{t-1})$ = $P(\epsilon_{t}>-(1-\rho)\mu -\rho z_{t-1} -\log{\alpha} + (1-\alpha)\log{k_{t}} |\hat{\theta},k_{t},z_{t-1})$

$P(r_{t}>1 |\hat{\theta},k_{t},z_{t-1})$ = $P(\epsilon_{t}>(1-\alpha)\log{k_{t}}+(\rho - 1)\mu -\rho z_{t-1} -\log{\alpha}|\hat{\theta},k_{t},z_{t-1})$

$P(r_{t}>1 |\hat{\theta},k_{t},z_{t-1})$ = $P(\epsilon_{t}>(1-\hat{\alpha})\log{k_{t}}+(\hat{\rho} - 1)\hat{\mu} -\hat{\rho} z_{t-1} -\log{\hat{\alpha}})$

$P(r_{t}>1 |\hat{\theta},k_{t},z_{t-1})$ =1 - $P(\epsilon_{t}\leq(1-\hat{\alpha})\log{k_{t}}+(\hat{\rho} - 1)\hat{\mu} -\hat{\rho} z_{t-1} -\log{\hat{\alpha}})$

Since $\epsilon_{t}$~$N(0,\hat{\sigma}^{2})$ and we know the values of $k_{t}$, $z_{t-1}$, $\hat{\alpha}$, $\hat{\rho}$, $\hat{\mu}$, and $\hat{\sigma}$, then we can use stats.norm.cdf to compute the probability.

In [222]:
k_current = 7500000
epsilon_t = (1 - alpha_mle) * np.log(k_current) - np.log(alpha_mle) + (rho_mle - 1) * mu_mle - 10 * rho_mle
prob = 1 - sts.norm.cdf(epsilon_t, loc=0, scale=sigma_mle)
print (prob)

1.0
